### TODO
1. Procent odpowiedzi poprawnych (accuracy) w zależności od parametru lambda.
2. Liczba różnych odpowiedzi (tzn. odp_uni != odp_bigram) w zależności od parametru lambda.
3. Tabele poprawności według kategorii dla każdej z wartości parametru lambda.

In [529]:
import pandas as pd
import re

Predykcji zmiennej celu dokonano przy użyciu następujących wartości parametru lambda, który mówi o tym jak bardzo ważny jest dla nas model dwugramowy a jak mało ważny jest model jednogramowy. Parametr `lambda` istotny jest jedynie z punktu widzenia modelu dwugramowego.

In [530]:
! ls ../ | grep test_set*

test_set_with_answers_no_lang_error_scaling_0.5_lambda.csv
test_set_with_answers_no_lang_error_scaling_0.66_lambda.csv
test_set_with_answers_no_lang_error_scaling_0.75_lambda.csv
test_set_with_answers_no_lang_error_scaling_0.95_lambda.csv
test_set_with_answers_no_lang_error_scaling_0.9_lambda.csv
test_set_with_answers_no_lang_error_scaling_1.0_lambda.csv
test_set_with_answers_no_lang_error_scaling.csv


In [531]:
file = 'test_set_with_answers_no_lang_error_scaling_0.95_lambda.csv'

In [532]:
df = pd.read_csv('../{}'.format(file))

Gdy w kolumnach `unigram_case` i/lub `bigram_case` znajduje się `NaN` oznacza to, że model kanału z błędem zwrócił dla analizowanej operacji `None`. `None` wiążę się z: 
 1. brakiem kandydatów (wygenerowane przez algorytm DL słowa nie znajdują się na liście istniejących słów języka polskiego) 
 2. błędem w zbiorze testowym określono ciąg jedynie zakazanych znaków (tzn. innych niż litery alfabetu łacińskiego rozszerzonego o polskie znaki diakrytyczne) - po przeprowadzeniu czyszczenia powstaje pusty napis, z którego nie można generować sensownych nowych słów
 3. wskazany błąd nie znajduje się w zdaniu
 
Przypadki 2 oraz 3 są rzadkie - stanowią nie więcej niż 30 obserwacji.

In [533]:
df.head()

,Unnamed: 0,text_with_error,corrected_text,is_valid_sentence,error,type,dist,category,file,basic_type_operation,nums,gold_standard,unigram_case,bigram_case
0,404,Milicja zaczęła przesłuchiwać wszystkich człon...,Milicja zaczęła przesłuchiwać wszystkich człon...,True,wydawnict,nonword,1,pisownia,plewic.08.0100.yaml,delete,"(9, 9)",wydawnictw,wydawnictw,wydawnictw
1,370,Guz olbrzymiokomórkowy kości (łac. tumor gigan...,Guz olbrzymiokomórkowy kości (łac. tumor gigan...,True,chrakteryzującym,nonword,1,pisownia,plewic.05.0014.yaml,delete,"(2, 2)",charakteryzującym,NaN,NaN
2,48,"Spośród możliwych populacji Evans, Lahn i inni...","Spośród możliwych populacji Evans, Lahn i inni...",True,wskzują,nonword,1,pisownia,plewic.06.0019.yaml,delete,"(3, 3)",wskazują,wskazują,wskazują
3,144,Pierwsza randka J.D. z Kailie nie przebiegła p...,Pierwsza randka J.D. z Kailie nie przebiegła p...,True,okazjii,nonword,1,pisownia,plewic.05.0086.yaml,insert,"(5, 6)",okazji,okazji,okazji
4,26,Posłowie na Sejm II Dubois Stanislaw,Posłowie na Sejm II Dubois Stanisław,False,Stanislaw,nonword,1,znaki diakrytyczne,plewic.01.0316.yaml,replace,"(6, 6)",stanisław,NaN,NaN


In [534]:
df_unigram_nan = df.loc[df['unigram_case'].isna()]

In [535]:
df_bigram_nan = df.loc[df['unigram_case'].isna()]

Obserwacje z `NaN` odpowiadają sobie w przypadku metody 1-gramów oraz 2-gramów:

In [536]:
set(df_unigram_nan.index) - set(df_bigram_nan.index)

set()

In [537]:
set(df_bigram_nan.index) - set(df_unigram_nan.index)

set()

Obserwacje, dla których model nie zwrócił odpowiedzi stanowią:

In [538]:
df_unigram_nan.shape[0] / df.shape[0]

0.06797933000712758

Aby poprawić ten wynik można by skorzystać z rozszerzonej listy słów dozwolonych w języku polskim. Jednocześnie na liście muszą znajdować się słowa istniejące, bez błędów, co wymaga kosztownej, manualnej pracy.

In [539]:
df_w_answer = df.dropna(subset=['unigram_case', 'bigram_case'])

In [540]:
df_w_answer.head()

,Unnamed: 0,text_with_error,corrected_text,is_valid_sentence,error,type,dist,category,file,basic_type_operation,nums,gold_standard,unigram_case,bigram_case
0,404,Milicja zaczęła przesłuchiwać wszystkich człon...,Milicja zaczęła przesłuchiwać wszystkich człon...,True,wydawnict,nonword,1,pisownia,plewic.08.0100.yaml,delete,"(9, 9)",wydawnictw,wydawnictw,wydawnictw
2,48,"Spośród możliwych populacji Evans, Lahn i inni...","Spośród możliwych populacji Evans, Lahn i inni...",True,wskzują,nonword,1,pisownia,plewic.06.0019.yaml,delete,"(3, 3)",wskazują,wskazują,wskazują
3,144,Pierwsza randka J.D. z Kailie nie przebiegła p...,Pierwsza randka J.D. z Kailie nie przebiegła p...,True,okazjii,nonword,1,pisownia,plewic.05.0086.yaml,insert,"(5, 6)",okazji,okazji,okazji
5,264,Generalny remont miał miejsce w 1908 roku z ok...,Generalny remont miał miejsce w 1908 roku z ok...,True,pokrytop,nonword,1,pisownia,plewic.05.0046.yaml,insert,"(6, 7)",pokryto,pokryto,pokryto
6,186,"Górna część Biłego Dunajca (ul. Batorego, ul. ...","Górna część Białego Dunajca (ul. Batorego, ul....",True,Biłego,nonword,1,pisownia,plewic.03.0060.yaml,delete,"(2, 2)",białego,byłego,byłego


### Poziom ogólny - cała ramka danych

Sprawdzam dla różnych wartości współczynnika lambda to czy metoda 1-gramowa oraz 2-gramowa zwracają różne wyniki:

In [541]:
def check_unigram_bigram_answer(csv_file):
    df = pd.read_csv('../{}'.format(csv_file))
    
    # ignore NaN
    df_w_answer = df.dropna(subset=['unigram_case', 'bigram_case'])
    
    return df_w_answer.loc[df_w_answer['unigram_case'] != df_w_answer['bigram_case']]

Model zwraca różne wyniki w zależności od użycia metody 1- bądź 2-gramów:

In [542]:
check_unigram_bigram_answer(file).shape

(26589, 14)

### 1-gram

Sprawdzam poprawność na zasadzie równości `'unigram_case'` z `gold_standard`:

In [543]:
df_correct = df_w_answer.loc[df_w_answer['gold_standard'] == df_w_answer['unigram_case']]

Metoda osiąga skuteczność:

In [544]:
df_correct.shape[0] / df_w_answer.shape[0]

0.8679200191189357

In [545]:
df_incorrect = df_w_answer.loc[df_w_answer['gold_standard'] != df_w_answer['unigram_case']]

Wprowadzam kolumnę, która informuje o poprawności odpowiedzi:

In [546]:
df_w_answer['is_correct'] = df_w_answer.apply(lambda x: int(x['gold_standard'] == x['unigram_case']), axis=1)

/home/pawelhdd/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [547]:
grouped_df = df_w_answer.groupby(['type', 'category', 'basic_type_operation']).agg({'is_correct': ['mean']}).sort_values(by=('is_correct', 'mean'), ascending=False)
grouped_df

is_correct
                                                                mean
type     category                    basic_type_operation           
nonword  znaki diakrytyczne          insert                 1.000000
realword znaki diakrytyczne/kontekst insert                 1.000000
nonword  znaki diakrytyczne          replace                0.986191
realword znaki diakrytyczne/kontekst replace                0.944625
nonword  znaki diakrytyczne          delete                 0.911111
         pisownia                    transpose              0.884644
                                     replace                0.848417
                                     insert                 0.837335
                                     delete                 0.784364
realword fleksja/liczba              replace                0.601261
         składnia                    replace                0.479326
                                     insert                 0.443983
         znaki diakrytyczne/kontekst delete                 0.441176
         fleksja/liczba              delete                 0.405325
         składnia                    delete                 0.272109
                                     transpose              0.186441
         fleksja/liczba              insert                 0.098039
                                     transpose              0.000000

In [548]:
df_w_answer.groupby(['type', 'category', 'basic_type_operation']).agg({'is_correct': ['mean']})

is_correct
                                                                mean
type     category                    basic_type_operation           
nonword  pisownia                    delete                 0.784364
                                     insert                 0.837335
                                     replace                0.848417
                                     transpose              0.884644
         znaki diakrytyczne          delete                 0.911111
                                     insert                 1.000000
                                     replace                0.986191
realword fleksja/liczba              delete                 0.405325
                                     insert                 0.098039
                                     replace                0.601261
                                     transpose              0.000000
         składnia                    delete                 0.272109
                                     insert                 0.443983
                                     replace                0.479326
                                     transpose              0.186441
         znaki diakrytyczne/kontekst delete                 0.441176
                                     insert                 1.000000
                                     replace                0.944625

In [549]:
grouped_df_type = df_w_answer.groupby(['type']).agg({'is_correct': ['mean']}).sort_values(by=('is_correct', 'mean'), ascending=False)
grouped_df_type

,is_correct
,mean
type,
nonword,0.886399
realword,0.740519


In [550]:
grouped_df_category = df_w_answer.groupby(['category']).agg({'is_correct': ['mean']}).sort_values(by=('is_correct', 'mean'), ascending=False)
grouped_df_category

,is_correct
,mean
category,
znaki diakrytyczne,0.986113
znaki diakrytyczne/kontekst,0.939388
pisownia,0.822071
fleksja/liczba,0.491734
składnia,0.352102


In [551]:
grouped_df_operation = df_w_answer.groupby(['basic_type_operation']).agg({'is_correct': ['mean']}).sort_values(by=('is_correct', 'mean'), ascending=False)
grouped_df_operation

,is_correct
,mean
basic_type_operation,
replace,0.932180
transpose,0.878258
insert,0.802075
delete,0.750816


Sprawdzam liczebności poszczególnych grup definiowanych przez różne wartości zmiennych kategorycznych:

In [552]:
grouped_df_type_category_count = df_w_answer.groupby(['type','category']).agg(['count'])['text_with_error'].sort_values(by='count', ascending=False)
grouped_df_type_category_count

count
type     category                          
nonword  pisownia                     66639
         znaki diakrytyczne           42990
realword znaki diakrytyczne/kontekst   9668
         fleksja/liczba                3569
         składnia                      2664

In [553]:
grouped_df_type_basic_type_op_count = df_w_answer.groupby(['type', 'basic_type_operation']).agg(['count'])['text_with_error'].sort_values(by='count', ascending=False)
grouped_df_type_basic_type_op_count

count
type     basic_type_operation       
nonword  replace               59119
         delete                28709
         insert                14996
realword replace               12586
nonword  transpose              6805
realword delete                 2248
         insert                 1005
         transpose                62

In [554]:
for i in range(grouped_df.shape[0]):
    print(re.sub("/| ", "_", " ".join(grouped_df.iloc[i].name)), grouped_df.iloc[i].values[0])

nonword_znaki_diakrytyczne_insert 1.0
realword_znaki_diakrytyczne_kontekst_insert 1.0
nonword_znaki_diakrytyczne_replace 0.9861909973686049
realword_znaki_diakrytyczne_kontekst_replace 0.9446247252172093
nonword_znaki_diakrytyczne_delete 0.9111111111111111
nonword_pisownia_transpose 0.884643644379133
nonword_pisownia_replace 0.8484174085064293
nonword_pisownia_insert 0.8373349339735895
nonword_pisownia_delete 0.7843636617359755
realword_fleksja_liczba_replace 0.6012605042016806
realword_składnia_replace 0.4793261868300153
realword_składnia_insert 0.44398340248962653
realword_znaki_diakrytyczne_kontekst_delete 0.4411764705882353
realword_fleksja_liczba_delete 0.40532544378698226
realword_składnia_delete 0.272108843537415
realword_składnia_transpose 0.1864406779661017
realword_fleksja_liczba_insert 0.09803921568627451
realword_fleksja_liczba_transpose 0.0


In [555]:
def create_filtered_dfs(grouped_df=grouped_df, df_to_filter=df_incorrect):
    """Return a dictionary of dataframes."""
    dfs_dict = dict()
    
    for i in range(grouped_df.shape[0]):
    
        # key indicates type, category and basic_type_operation
        col_tuple = grouped_df.iloc[i].name
        key = re.sub("/| ", "_", " ".join(col_tuple))

        # value is a dataframe filtered with key elements
        mask = (df_to_filter['type'] == col_tuple[0]) & (df_to_filter['category'] == col_tuple[1]) & (df_to_filter['basic_type_operation'] == col_tuple[2])

        # get percentage of correct answers for each category
        percentage = grouped_df.iloc[i].values[0]

        dfs_dict[key] = (df_to_filter[mask], percentage)
        
    return dfs_dict

In [556]:
dataframes_filtered_dict = create_filtered_dfs()

In [557]:
dataframes_filtered_dict['realword_składnia_transpose'][0].head(10)

,Unnamed: 0,text_with_error,corrected_text,is_valid_sentence,error,type,dist,category,file,basic_type_operation,nums,gold_standard,unigram_case,bigram_case
119,349,W biegu na 15 km jego starta do Thomasa Wassbe...,W biegu na 15 km jego strata do Thomasa Wassbe...,True,starta,realword,1,składnia,plewic.08.0044.yaml,transpose,"(3, 2)",strata,startą,startą
840,79,"Jest wieloletnim graczem Athleticu Bilbao, dal...","Jest wieloletnim graczem Athleticu Bilbao, dla...",True,dal,realword,1,składnia,plewic.09.0124.yaml,transpose,"(2, 1)",dla,dał,dał
1173,94,Współpracował z burmistrzem Janem Rajchlem w k...,Współpracował z burmistrzem Janem Rajchlem w k...,True,palny,realword,1,składnia,plewic.09.0042.yaml,transpose,"(2, 1)",plany,palmy,palnę
3466,345,"Współpracując z nadajnikami radiowymi, zapewni...","Współpracując z nadajnikami radiowymi, zapewni...",True,kliku,realword,1,składnia,plewic.04.0037.yaml,transpose,"(2, 1)",kilku,pliku,gliku
3906,288,Dal większej dokładności pomiaru skonstruowano...,Dla większej dokładności pomiaru skonstruowano...,True,Dal,realword,1,składnia,plewic.09.0045.yaml,transpose,"(2, 1)",dla,dał,dał
5779,11,"! 'Rozdaje pomnażania: schizotomia, schizogoni...","! 'Rodzaje pomnażania: schizotomia, schizogoni...",True,Rozdaje,realword,1,składnia,plewic.04.0039.yaml,transpose,"(3, 2)",rodzaje,rozdaję,rozdaję
7768,248,Jest jednym z kliku smartphonów pozwalających ...,Jest jednym z kilku smartphonów pozwalających ...,True,kliku,realword,1,składnia,plewic.04.0088.yaml,transpose,"(2, 1)",kilku,pliku,gliku
12413,318,Wówczas na lewicy doszło do znacznych kłótni p...,Wówczas na lewicy doszło do znacznych kłótni p...,True,strać,realword,1,składnia,plewic.09.0117.yaml,transpose,"(3, 2)",starć,straż,strac
13031,218,Imigranci ci byli z reguły zamieszani w konfli...,Imigranci ci byli z reguły zamieszani w konfli...,True,tel,realword,1,składnia,plewic.05.0063.yaml,transpose,"(2, 1)",tle,tej,teł
17020,240,Pokrywa śniegu w listopadzie miała około 20 cm...,Pokrywa śniegu w listopadzie miała około 20 cm...,True,morzy,realword,1,składnia,plewic.01.0358.yaml,transpose,"(2, 1)",mrozy,morzu,morzę


### 2-gram

Sprawdzam poprawność `bigram_case` na zasadzie równości z `gold_standard`:

In [558]:
df_correct = df_w_answer.loc[df_w_answer['gold_standard'] == df_w_answer['bigram_case']]

Metoda osiąga skuteczność:

In [559]:
df_correct.shape[0] / df_w_answer.shape[0]

0.7256512387477098

In [560]:
df_incorrect = df_w_answer.loc[df_w_answer['gold_standard'] != df_w_answer['bigram_case']]

Wprowadzam kolumnę, która informuje o poprawności odpowiedzi:

In [561]:
df_w_answer['is_correct'] = df_w_answer.apply(lambda x: int(x['gold_standard'] == x['bigram_case']), axis=1)

/home/pawelhdd/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [562]:
grouped_df = df_w_answer.groupby(['type', 'category', 'basic_type_operation']).agg({'is_correct': ['mean']}).sort_values(by=('is_correct', 'mean'), ascending=False)
grouped_df

is_correct
                                                                mean
type     category                    basic_type_operation           
nonword  znaki diakrytyczne          insert                 1.000000
realword znaki diakrytyczne/kontekst insert                 1.000000
nonword  znaki diakrytyczne          replace                0.855646
                                     delete                 0.822222
realword znaki diakrytyczne/kontekst replace                0.783942
nonword  pisownia                    transpose              0.782219
                                     insert                 0.740630
                                     replace                0.709755
                                     delete                 0.608603
realword fleksja/liczba              replace                0.389916
         znaki diakrytyczne/kontekst delete                 0.372549
         fleksja/liczba              delete                 0.252959
         składnia                    replace                0.209801
                                     insert                 0.182573
         fleksja/liczba              insert                 0.047059
         składnia                    delete                 0.044218
                                     transpose              0.016949
         fleksja/liczba              transpose              0.000000

In [563]:
grouped_df_gen = df_w_answer.groupby(['type', 'category', 'basic_type_operation']).agg({'is_correct': ['mean']})

In [564]:
grouped_df_gen

is_correct
                                                                mean
type     category                    basic_type_operation           
nonword  pisownia                    delete                 0.608603
                                     insert                 0.740630
                                     replace                0.709755
                                     transpose              0.782219
         znaki diakrytyczne          delete                 0.822222
                                     insert                 1.000000
                                     replace                0.855646
realword fleksja/liczba              delete                 0.252959
                                     insert                 0.047059
                                     replace                0.389916
                                     transpose              0.000000
         składnia                    delete                 0.044218
                                     insert                 0.182573
                                     replace                0.209801
                                     transpose              0.016949
         znaki diakrytyczne/kontekst delete                 0.372549
                                     insert                 1.000000
                                     replace                0.783942

In [565]:
grouped_df_gen[('is_correct', 'mean')]

type      category                     basic_type_operation
nonword   pisownia                     delete                  0.608603
                                       insert                  0.740630
                                       replace                 0.709755
                                       transpose               0.782219
          znaki diakrytyczne           delete                  0.822222
                                       insert                  1.000000
                                       replace                 0.855646
realword  fleksja/liczba               delete                  0.252959
                                       insert                  0.047059
                                       replace                 0.389916
                                       transpose               0.000000
          składnia                     delete                  0.044218
                                       insert                  0.182573
    

In [566]:
grouped_df_type = df_w_answer.groupby(['type']).agg({'is_correct': ['mean']}).sort_values(by=('is_correct', 'mean'), ascending=False)
grouped_df_type

,is_correct
,mean
type,
nonword,0.749227
realword,0.563109


In [567]:
grouped_df_category = df_w_answer.groupby(['category']).agg({'is_correct': ['mean']}).sort_values(by=('is_correct', 'mean'), ascending=False)
grouped_df_category

,is_correct
,mean
category,
znaki diakrytyczne,0.855618
znaki diakrytyczne/kontekst,0.779892
pisownia,0.680592
fleksja/liczba,0.314654
składnia,0.109234


In [568]:
grouped_df_operation = df_w_answer.groupby(['basic_type_operation']).agg({'is_correct': ['mean']}).sort_values(by=('is_correct', 'mean'), ascending=False)
grouped_df_operation

,is_correct
,mean
basic_type_operation,
replace,0.791842
transpose,0.775302
insert,0.701956
delete,0.573570


Sprawdzam liczebności poszczególnych grup definiowanych przez różne wartości zmiennych kategorycznych:

In [569]:
grouped_df_type_category_count = df_w_answer.groupby(['type','category']).agg(['count'])['text_with_error'].sort_values(by='count', ascending=False)
grouped_df_type_category_count

count
type     category                          
nonword  pisownia                     66639
         znaki diakrytyczne           42990
realword znaki diakrytyczne/kontekst   9668
         fleksja/liczba                3569
         składnia                      2664

In [570]:
grouped_df_type_basic_type_op_count = df_w_answer.groupby(['type', 'basic_type_operation']).agg(['count'])['text_with_error'].sort_values(by='count', ascending=False)
grouped_df_type_basic_type_op_count

count
type     basic_type_operation       
nonword  replace               59119
         delete                28709
         insert                14996
realword replace               12586
nonword  transpose              6805
realword delete                 2248
         insert                 1005
         transpose                62

In [571]:
for i in range(grouped_df.shape[0]):
    print(re.sub("/| ", "_", " ".join(grouped_df.iloc[i].name)), grouped_df.iloc[i].values[0])

nonword_znaki_diakrytyczne_insert 1.0
realword_znaki_diakrytyczne_kontekst_insert 1.0
nonword_znaki_diakrytyczne_replace 0.8556458561348764
nonword_znaki_diakrytyczne_delete 0.8222222222222222
realword_znaki_diakrytyczne_kontekst_replace 0.7839422171045745
nonword_pisownia_transpose 0.7822189566495225
nonword_pisownia_insert 0.7406295851674003
nonword_pisownia_replace 0.7097551928783383
nonword_pisownia_delete 0.6086031258721741
realword_fleksja_liczba_replace 0.3899159663865546
realword_znaki_diakrytyczne_kontekst_delete 0.37254901960784315
realword_fleksja_liczba_delete 0.2529585798816568
realword_składnia_replace 0.20980091883614088
realword_składnia_insert 0.1825726141078838
realword_fleksja_liczba_insert 0.047058823529411764
realword_składnia_delete 0.04421768707482993
realword_składnia_transpose 0.01694915254237288
realword_fleksja_liczba_transpose 0.0


In [572]:
def create_filtered_dfs(grouped_df=grouped_df, df_to_filter=df_incorrect):
    """Return a dictionary of dataframes."""
    dfs_dict = dict()
    
    for i in range(grouped_df.shape[0]):
    
        # key indicates type, category and basic_type_operation
        col_tuple = grouped_df.iloc[i].name
        key = re.sub("/| ", "_", " ".join(col_tuple))

        # value is a dataframe filtered with key elements
        mask = (df_to_filter['type'] == col_tuple[0]) & (df_to_filter['category'] == col_tuple[1]) & (df_to_filter['basic_type_operation'] == col_tuple[2])

        # get percentage of correct answers for each category
        percentage = grouped_df.iloc[i].values[0]

        dfs_dict[key] = (df_to_filter[mask], percentage)
        
    return dfs_dict

In [573]:
dataframes_filtered_dict = create_filtered_dfs()

In [574]:
dataframes_filtered_dict['realword_składnia_transpose'][0].head(10)

,Unnamed: 0,text_with_error,corrected_text,is_valid_sentence,error,type,dist,category,file,basic_type_operation,nums,gold_standard,unigram_case,bigram_case,is_correct
119,349,W biegu na 15 km jego starta do Thomasa Wassbe...,W biegu na 15 km jego strata do Thomasa Wassbe...,True,starta,realword,1,składnia,plewic.08.0044.yaml,transpose,"(3, 2)",strata,startą,startą,0
840,79,"Jest wieloletnim graczem Athleticu Bilbao, dal...","Jest wieloletnim graczem Athleticu Bilbao, dla...",True,dal,realword,1,składnia,plewic.09.0124.yaml,transpose,"(2, 1)",dla,dał,dał,0
1173,94,Współpracował z burmistrzem Janem Rajchlem w k...,Współpracował z burmistrzem Janem Rajchlem w k...,True,palny,realword,1,składnia,plewic.09.0042.yaml,transpose,"(2, 1)",plany,palmy,palnę,0
3466,345,"Współpracując z nadajnikami radiowymi, zapewni...","Współpracując z nadajnikami radiowymi, zapewni...",True,kliku,realword,1,składnia,plewic.04.0037.yaml,transpose,"(2, 1)",kilku,pliku,gliku,0
3906,288,Dal większej dokładności pomiaru skonstruowano...,Dla większej dokładności pomiaru skonstruowano...,True,Dal,realword,1,składnia,plewic.09.0045.yaml,transpose,"(2, 1)",dla,dał,dał,0
5779,11,"! 'Rozdaje pomnażania: schizotomia, schizogoni...","! 'Rodzaje pomnażania: schizotomia, schizogoni...",True,Rozdaje,realword,1,składnia,plewic.04.0039.yaml,transpose,"(3, 2)",rodzaje,rozdaję,rozdaję,0
7768,248,Jest jednym z kliku smartphonów pozwalających ...,Jest jednym z kilku smartphonów pozwalających ...,True,kliku,realword,1,składnia,plewic.04.0088.yaml,transpose,"(2, 1)",kilku,pliku,gliku,0
12413,318,Wówczas na lewicy doszło do znacznych kłótni p...,Wówczas na lewicy doszło do znacznych kłótni p...,True,strać,realword,1,składnia,plewic.09.0117.yaml,transpose,"(3, 2)",starć,straż,strac,0
12862,239,miejscowość i wyjechał do U.S.A. Ale on tam je...,miejscowość i wyjechał do U.S.A. Ale on tam je...,False,tka,realword,1,składnia,plewic.03.0043.yaml,transpose,"(2, 1)",tak,tak,ćka,1
13031,218,Imigranci ci byli z reguły zamieszani w konfli...,Imigranci ci byli z reguły zamieszani w konfli...,True,tel,realword,1,składnia,plewic.05.0063.yaml,transpose,"(2, 1)",tle,tej,teł,0
